In [1]:
import torch.nn as nn
import torch

In [2]:
class TextCNN(nn.Module):
    def __init__(self, config):
        super(TextCNN, self).__init__()
        self.dropout_rate = config.dropout_rate
        self.num_class = config.num_clas
 
        self.embedding = nn.Embedding(num_embeddings=config.vocab_size, 
                                embedding_dim=config.embedding_size)  
        self.convs = nn.ModuleList([
                       nn.Sequential(
                			  nn.Conv1d(in_channels=config.embedding_size,
                			  out_channels=config.out_channels, 
                              kernel_size= ks),  						
                              nn.ReLU(), 
                              nn.MaxPool1d(kernel_size=config.max_len-ks+1)) 
                       for ks in config.kernel_size ]) # 创建3个nn.Sequential，包含了 图中的convolution层、activation function层 和 maxPooling层, 其中每个层的参数都有变化
        self.fc = nn.Linear(in_features=config.out_channels*len(config.kernel_size),
                            out_features=config.num_class) # 每种类别的卷积核个数相乘，得到的长度就是全连接层输入的长度 
    
    def forward(self, x):
        embed_x = self.embedding(x) # b x src_len
        embed_x = embed_x.permute(0, 2, 1) 
        # b x src_len x embed_size --> b x embed_size x src_lem
        out = [conv(embed_x) for conv in self.convs]  #计算每层卷积的结果，这里输出的结果已经经过池化层处理了
        out = torch.cat(out, dim=1)  # 对池化后的向量进行拼接
        out = out.view(-1, out.size(1)) # 拉成一竖条作为全连接层的输入
        out = F.dropout(input=out, p=self.dropout_rate) # 这里也没有在图中的表现出来，这里是随机让一部分的神经元失活，避免过拟合。它只会在train的状态下才会生效。进入train状态可查看nn.Module。train()方法
        out = self.fc(out)
        return out